# MNIST FPGA Implementation
Joash Naidoo

Tutorial on creating synthesizable FPGA code for classifying the Hello World of Neural Networks, the MNIST data set. Typically I enjoy implementing things from first principles however hls4ml requires Keras model (tensorflow backend), so that is what I'll be using.

This project was realized with the help of the following resources:

1. [Keras Tutorial](https://aigeekprogrammer.com/keras-python-mnist-handwritten-digit-recognition/)

2. [Keras API](https://keras.io/api/models/)

3. [Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/chap1.html)

4. [hls4ml](https://fastmachinelearning.org/hls4ml/)

5. [hls4ml Tutorial](https://keras.io/api/models/)

In [1]:
# Imports
import numpy as np
import keras

# Offers additional methods specific to MNIST dataset
from keras.datasets import mnist

In [2]:
# Load MNIST Data
(train_in, train_lbl), (test_in, test_lbl) = mnist.load_data()

In [3]:
# Flatten 28x28 2D image
train_in = train_in.reshape((-1, (28*28)))
test_in = test_in.reshape((-1, (28*28)))

In [4]:
# Create Model

# Adopting basic model from http://neuralnetworksanddeeplearning.com/chap1.html

from keras.models import Sequential
from keras.layers import Dense

model = Sequential ([
        Dense(30, activation='sigmoid', input_shape=(784,)), # input layer to 30 hidden
        Dense(10, activation='sigmoid') # 10 output
        #Dense(10, activation='sigmoid') # 10 ouput neurons
])

# Compile the model

model.compile(
    optimizer='SGD',
    loss='mean_squared_error',
    metrics=['accuracy']
)

In [5]:
# Train the model!

# Gives about a 91% accuracy. Not the main focus of this project

model.fit(
    x=train_in,
    y=keras.utils.to_categorical(train_lbl),
    epochs=30,
    batch_size=10,
    shuffle=True
)

Epoch 1/30
6000/6000 [==============================] - 5s 882us/step - loss: 0.0884 - accuracy: 0.4006
Epoch 2/30
6000/6000 [==============================] - 5s 877us/step - loss: 0.0672 - accuracy: 0.6087
Epoch 3/30
6000/6000 [==============================] - 5s 875us/step - loss: 0.0570 - accuracy: 0.7005
Epoch 4/30
6000/6000 [==============================] - 5s 867us/step - loss: 0.0502 - accuracy: 0.7496
Epoch 5/30
6000/6000 [==============================] - 5s 868us/step - loss: 0.0451 - accuracy: 0.7931
Epoch 6/30
6000/6000 [==============================] - 5s 872us/step - loss: 0.0409 - accuracy: 0.8278
Epoch 7/30
6000/6000 [==============================] - 5s 888us/step - loss: 0.0374 - accuracy: 0.8501
Epoch 8/30
6000/6000 [==============================] - 5s 862us/step - loss: 0.0345 - accuracy: 0.8623
Epoch 9/30
6000/6000 [==============================] - 6s 960us/step - loss: 0.0320 - accuracy: 0.8684
Epoch 10/30
6000/6000 [==============================] - 6s 1ms/

In [6]:
# Evaluate the model

eval = model.evaluate(test_in, keras.utils.to_categorical(test_lbl))
eval

313/313 [==============================] - 0s 1ms/step - loss: 0.0180 - accuracy: 0.9074


[0.01795855723321438, 0.9074000120162964]

## HLS4ML

In [7]:
import hls4ml

from keras.utils.vis_utils import plot_model

%matplotlib inline

In [8]:
# Need a clearer idea what this is for

config = hls4ml.utils.config_from_keras_model(model, granularity='model')
config

Interpreting Sequential
Topology:
Layer name: dense_input, layer type: Input
Layer name: dense, layer type: Dense
  -> Activation (sigmoid), layer name: dense
Layer name: dense_1, layer type: Dense
  -> Activation (sigmoid), layer name: dense_1


{'Model': {'Precision': 'ap_fixed<16,6>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'}}

In [9]:
# Building the model for Kintex7 KC705 Evaluation board

hls_model = hls4ml.converters.convert_from_keras_model(
                    model,
                    hls_config=config,
                    output_dir='model_1\hls4ml_prj',
                    fpga_part='XC7K325T-2FFG900C'
)

Interpreting Sequential
Topology:
Layer name: dense_input, layer type: InputLayer, current shape: [[None, 784]]
Layer name: dense, layer type: Dense, current shape: [[None, 784]]
Layer name: dense_1, layer type: Dense, current shape: [[None, 30]]
Creating HLS model


In [10]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\joash\\AppData\\Local\\Temp\\tmpjji0t7po.png'

In [12]:
import os
# Add Vivado to PATH
print(os.environ['PATH'])
os.environ['PATH'] += ';C:\\Windows\\System32\\'
os.environ['PATH'] += ';C:\\Xilinx\\Vivado\\2019.2\\bin\\unwrapped\\'
print('\n'+os.environ['PATH'])

hls_model.compile()
hls_output = hls_model.predict(test_in)

C:\Users\joash\miniconda3\envs\mnist_fpga;C:\Users\joash\miniconda3\envs\mnist_fpga\Library\mingw-w64\bin;C:\Users\joash\miniconda3\envs\mnist_fpga\Library\usr\bin;C:\Users\joash\miniconda3\envs\mnist_fpga\Library\bin;C:\Users\joash\miniconda3\envs\mnist_fpga\Scripts;C:\Users\joash\miniconda3\envs\mnist_fpga\bin;C:\Users\joash\miniconda3\condabin;C:\ProgramData\Oracle\Java\javapath;C:\Program Files\National Instruments\Shared\OpenVINO;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files (x86)\IVI Foundation\VISA\WinNT\Bin;C:\Program Files\IVI Foundation\VISA\Win64\Bin;C:\Program Files (x86)\IVI Foundation\VISA\WinNT\Bin;C:\Program Files\Git\cmd;C:\Program Files\Neovim\bin;C:\Program Files\Inkscape\bin;C:\Program Files (x86)\ZeroTier\One;C:\Users\joash\AppData\Local\Programs\Python\Python38\Scripts;C:\Users\joash\AppData\Local\Programs\Python\Python38;C:\Users\joash\AppData\Local\Microsoft\Windows

Exception: Failed to compile project "myproject"